In [4]:
import requests
from psycopg2 import connect, sql

# Configure your PostgreSQL connection string
conn_string = "dbname='etl_bites' user='ilhaam.ahmed' password='etl_proj' host='localhost' port='5432'"

def get_data_from_api(url):
    response = requests.get(url)
    return response.json()

posts_url = "https://jsonplaceholder.typicode.com/posts"
users_url = "https://jsonplaceholder.typicode.com/users"

posts_data = get_data_from_api(posts_url)
users_data = get_data_from_api(users_url)

explain.....

In [5]:
def join_posts_and_users(posts, users):
    for post in posts:
        for user in users:
            if post['userId'] == user['id']:
                post['author'] = user['name']
    return posts

combined_data = join_posts_and_users(posts_data, users_data)

explain...

In [6]:
# Create tables in analytical DB
# This could also be done manually via a GUI (e.g. TablePlus) or with a SQL script
def execute_query_postgresql(conn_string, query):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_api_data_table = '''
CREATE TABLE api_data (
    post_id INTEGER NOT NULL,
    title TEXT NOT NULL,
    body TEXT NOT NULL,
    user_id INTEGER NOT NULL,
    author TEXT NOT NULL
);
'''

execute_query_postgresql(conn_string, create_api_data_table)

In [7]:
def insert_data_to_postgresql(conn_string, table_name, data):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            for item in data:
                query = sql.SQL("INSERT INTO {} (post_id, title, body, user_id, author) VALUES (%s, %s, %s, %s, %s)").format(sql.Identifier(table_name))
                cur.execute(query, (item['id'], item['title'], item['body'], item['userId'], item['author']))
        conn.commit()

table_name = "api_data"
insert_data_to_postgresql(conn_string, table_name, combined_data)